<a href="https://colab.research.google.com/github/zihanzhao1029/ML_project_HW3/blob/main/ML_project_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Github link: 

##1. Discuss the dataset in general terms and describe why building a predictive model using this data might be practically useful.  Who could benefit from a model like this? Explain.

### Stanford Sentiment Treebank dataset(SST) consists of movie reviews and its corresponding label 0(negative) and 1(postive). Building a predictive model using this dataset can be beneficial for movie production firms, movie critics, and audience rating websites like Rotten Tomatoes to accurately classify movie reviews into positive or negative sentiment categories. In this case, movie prodution firms can update their idea for movies based on audience feedback, and professional movie critics can readily write movie review report.Similarly, the movie rating website can update improve their recommended system based on this predictive model

##2. Run at least three prediction models to try to predict the SST sentiment dataset well.

In [ ]:
#install aimodelshare library
! pip install aimodelshare==0.0.189

In [1]:
# Get Stanford Sentiment Treebank dataset
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/sst2_competition_data-repository:latest') 


Data downloaded successfully.


In [2]:
# Set up X_train, X_test, and y_train_labels objects
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=Warning)

X_train=pd.read_csv("sst2_competition_data/X_train.csv", squeeze=True)
X_test=pd.read_csv("sst2_competition_data/X_test.csv", squeeze=True)

y_train_labels=pd.read_csv("sst2_competition_data/y_train_labels.csv", squeeze=True)

# ohe encode Y data
y_train = pd.get_dummies(y_train_labels)

X_train.head()

0    The Rock is destined to be the 21st Century 's...
1    The gorgeously elaborate continuation of `` Th...
2    Singer/composer Bryan Adams contributes a slew...
3                 Yet the act is still charming here .
4    Whether or not you 're enlightened by any of D...
Name: text, dtype: object

In [ ]:
y_train.shape
print(y_train)

      Negative  Positive
0            0         1
1            0         1
2            0         1
3            0         1
4            0         1
...        ...       ...
6915         1         0
6916         1         0
6917         0         1
6918         1         0
6919         1         0

[6920 rows x 2 columns]


In [3]:
# Preprocessing dataset use tf.keras tokenizer

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import numpy as np

# Build vocabulary from training text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

# preprocessor tokenizes words and makes sure all documents have the same length
def preprocessor(data, maxlen=40, max_words=10000):

    sequences = tokenizer.texts_to_sequences(data)

    word_index = tokenizer.word_index
    X = pad_sequences(sequences, maxlen=maxlen)

    return X

print(preprocessor(X_train).shape)
print(preprocessor(X_test).shape)

(6920, 40)
(1821, 40)


### Use an Embedding layer and LSTM layers in at least one model

In [ ]:
#model version 144 in leaderboard
# fit model on preprocessed data and save preprocessor function and model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten

model = Sequential()
model.add(Embedding(10000, 16, input_length=40))
model.add(LSTM(64, return_sequences=True, dropout=0.1))
model.add(LSTM(128, dropout=0.1))
model.add(Flatten())
model.add(Dense(2, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
173/173 [==============================] - 67s 386ms/step - loss: 0.6466 - acc: 0.6315 - val_loss: 0.8242 - val_acc: 0.4429
Epoch 2/10
173/173 [==============================] - 68s 395ms/step - loss: 0.4685 - acc: 0.7822 - val_loss: 0.7096 - val_acc: 0.6004
Epoch 3/10
173/173 [==============================] - 67s 387ms/step - loss: 0.3534 - acc: 0.8438 - val_loss: 0.5999 - val_acc: 0.7276
Epoch 4/10
173/173 [==============================] - 72s 418ms/step - loss: 0.2834 - acc: 0.8822 - val_loss: 0.4833 - val_acc: 0.7883
Epoch 5/10
173/173 [==============================] - 72s 415ms/step - loss: 0.2312 - acc: 0.9010 - val_loss: 0.8147 - val_acc: 0.6337
Epoch 6/10
173/173 [==============================] - 67s 385ms/step - loss: 0.1973 - acc: 0.9214 - val_loss: 0.5132 - val_acc: 0.7905
Epoch 7/10
173/173 [==============================] - 73s 421ms/step - loss: 0.1776 - acc: 0.9290 - val_loss: 0.9073 - val_acc: 0.6777
Epoch 8/10
173/173 [==============================] - 7

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 40, 16)            160000    
                                                                 
 lstm_2 (LSTM)               (None, 40, 64)            20736     
                                                                 
 lstm_3 (LSTM)               (None, 128)               98816     
                                                                 
 flatten_5 (Flatten)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 2)                 258       
                                                                 
Total params: 279,810
Trainable params: 279,810
Non-trainable params: 0
_________________________________________________________________


### Use an Embedding layer and Conv1d layers in at least one model

In [4]:
#model version 145 in leaderboard
from tensorflow.keras.layers import Conv1D, Dense, Embedding, GlobalMaxPooling1D, MaxPooling1D,Flatten
from tensorflow.keras.models import Sequential

model2 = Sequential()
model2.add(Embedding(10000, 16, input_length=40))
model2.add(Conv1D(64, 5, activation='relu'))
model2.add(MaxPooling1D(5))
model2.add(Conv1D(128, 5, activation='relu'))
model2.add(GlobalMaxPooling1D())
model2.add(Flatten())
model2.add(Dense(2, activation='sigmoid'))
model2.summary()

model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model2.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 16)            160000    
                                                                 
 conv1d (Conv1D)             (None, 36, 64)            5184      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 7, 64)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 3, 128)            41088     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 flatten (Flatten)           (None, 128)               0

### Use transfer learning with glove embeddings for at least one of these models



In [ ]:
#check the number of training sample
print(X_train.shape)
print(y_train.shape)

(6920,)
(6920, 2)


In [5]:
# Tokenize the text data into one hot vectors and use it for embedding matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 40  # We will cut reviews after 40 words in sequence
training_samples = 5000  # We will be training on 5000 samples
validation_samples = 1920  # We will be validating on 1920 samples
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)


sequences = tokenizer.texts_to_sequences(X_train) # converts words in each text to each word's numeric index in tokenizer dictionary.

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(y_train)

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples] #100 words
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Found 13835 unique tokens.
Shape of data tensor: (6920, 40)
Shape of label tensor: (6920, 2)


In [ ]:
print(y_val.shape)
print(x_val.shape)

(1920, 2)
(1920, 40)


In [6]:
# use glove embedding matrix
# What if we wanted to use a matrix of pretrained embeddings?  Same as transfer learning before, but now we are importing a pretrained Embedding matrix:
# Download Glove embedding matrix weights (Might take 10 mins or so!)
! wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2023-04-15 18:33:37--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2023-04-15 18:33:37--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2023-04-15 18:33:37--  https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [app

In [7]:
! unzip glove.6B.zip 

Archive:  glove.6B.zip
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        


In [8]:
# Extract embedding data for 100 feature embedding matrix
import os
glove_dir = os.getcwd()

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400001 word vectors.


In [9]:
# Build embedding matrix
embedding_dim =100 # number of features
max_words=10000 #maximum number of words to be used in the vocabulary

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [12]:
# model version 163 in leaderboard
# Set up model architecture used before and then import Glove weights to Embedding layer:
# use embedding layer with conv1D layer model

model2 = Sequential()
model2.add(Embedding(10000, 100, input_length=40)) # change the embedding dim to be consistent with embedding_inex in matrix
model2.add(Conv1D(64, 5, activation='relu'))
model2.add(MaxPooling1D(5))
model2.add(Conv1D(128, 5, activation='relu'))
model2.add(GlobalMaxPooling1D())
model2.add(Flatten())
model2.add(Dense(2, activation='sigmoid'))
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 40, 100)           1000000   
                                                                 
 conv1d_4 (Conv1D)           (None, 36, 64)            32064     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 7, 64)            0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 3, 128)            41088     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 flatten_2 (Flatten)         (None, 128)              

In [13]:
# Add weights in same manner as transfer learning and turn of trainable option before fitting model to freeze weights.
model2.layers[0].set_weights([embedding_matrix])
model2.layers[0].trainable = False



model2.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model2.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model2.save_weights('pre_trained_glove_model.h5')

Epoch 1/10
157/157 [==============================] - 5s 24ms/step - loss: 0.6344 - acc: 0.6422 - val_loss: 0.5646 - val_acc: 0.7016
Epoch 2/10
157/157 [==============================] - 2s 13ms/step - loss: 0.5091 - acc: 0.7468 - val_loss: 0.7443 - val_acc: 0.6281
Epoch 3/10
157/157 [==============================] - 2s 14ms/step - loss: 0.4426 - acc: 0.7960 - val_loss: 0.5181 - val_acc: 0.7370
Epoch 4/10
157/157 [==============================] - 2s 14ms/step - loss: 0.3646 - acc: 0.8362 - val_loss: 0.5764 - val_acc: 0.7182
Epoch 5/10
157/157 [==============================] - 2s 13ms/step - loss: 0.2976 - acc: 0.8748 - val_loss: 0.5641 - val_acc: 0.7437
Epoch 6/10
157/157 [==============================] - 3s 17ms/step - loss: 0.2330 - acc: 0.9074 - val_loss: 0.6178 - val_acc: 0.7422
Epoch 7/10
157/157 [==============================] - 3s 22ms/step - loss: 0.1846 - acc: 0.9316 - val_loss: 0.7343 - val_acc: 0.7281
Epoch 8/10
157/157 [==============================] - 3s 22ms/step - 

In [ ]:
# Evaluate the model performance using val set
model2.load_weights('pre_trained_glove_model.h5')
model2.evaluate(x_val, y_val)

60/60 [==============================] - 1s 16ms/step - loss: 1.5401 - acc: 0.6724


[1.540058970451355, 0.6723958253860474]

## 3.Team discussion via slack

After team disscussion , i update the model 2(embedding layyers with conv1d layer) by adding one more conv1d layers with 128 neurons), and i also update the transfer learning by add two more conv1d layers with 128,256 neurons separately.


##4. Discuss which models you tried and which models performed better and point out relevant hyper-parameter values for successful models.

I run the following model:

Embedding layer and two LSTM layers with 64,128 neurons separately

Embedding layer and two Conv1d layers with 64,128 neurons separately

transfer learning with glove embeddings on my  second model

Embedding layer and four Conv1d layers with 64,64,128,128 neurons separately

transfer learning with glove embeddings on my  second model with two more Conv1d layer with 128,256 separately.

In this case, the transfer learning with four Conv1d layers plus glove embeddings performs better, the relevant parameters are :relu activation,softmax as final output activation,optimizer using rmsprop,loss_function using binary_crosstropy.
